In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
processed_data_path = Path("../data/processed/consumo_tarifas_meses.xlsx").resolve()

data = pd.read_excel(processed_data_path)
data

In [ ]:
# Ajustar formato da columa Meses
data["Meses"] = data["Meses"].apply(lambda x: int(str(x).split(".")[0]))

In [ ]:
data["Item Faturamento BW"] = data["Item Faturamento BW"].apply(lambda x: str(x).replace(" ",""))
for i, item in enumerate(np.sort(data["Item Faturamento BW"].unique())):
    print(f"{i+1:02d} - {item}")


In [ ]:
data["SubAgrupador de Fatu"] = data["SubAgrupador de Fatu"].apply(lambda x: str(x).replace(" ",""))
for i, item in enumerate(np.sort(data["SubAgrupador de Fatu"].unique())):
    print(f"{i+1:02d} - {item}")


In [ ]:
data["grupo"] = data["grupo"].apply(lambda x: str(x).replace(" ",""))
for i, item in enumerate(np.sort(data["grupo"].unique())):
    print(f"{i+1:02d} - {item}")

In [ ]:
data = data.fillna(0)

In [ ]:
index = [
    'Empresa',
    'Nº instalação',
    "Meses"
    ]

columns = "Item Faturamento BW"
value = "Valores"

data_piv = pd.pivot_table(data, values=value, index=index, columns=columns)

data_piv = data_piv.reset_index()

data_piv["EnergAtvInj.mUCoPT-FPTE"] = data_piv["EnergAtvInj.mUCoPT-FPTE"] + data_piv["EnergAtvInj.mUCoPT-FP-TE"]
data_piv = data_piv.drop("EnergAtvInj.mUCoPT-FP-TE", axis=1)
data_piv



# Extração de características

In [ ]:
data_feat = pd.DataFrame()
keys = ["Nº instalação", "SubAgrupador de Fatu"]

for (cliente, grupo), grouped_df in data.groupby(by=keys):
    
    keys_2 = keys.copy()
    keys_2.append("Meses")
    df_tmp = pd.DataFrame()
    df_tmp["cliente"] = cliente,
    df_tmp["grupo"] = grupo,
    df_tmp["count"] = grouped_df["Meses"].unique().shape[0],
    df_tmp["last"] = grouped_df["Meses"].max(),
    df_tmp["mean"] = float(round(grouped_df.groupby(keys_2).sum()["Valores"].mean(),4)),
    df_tmp["max"] = float(round(grouped_df.groupby(keys_2).sum()["Valores"].max(),4)),
    df_tmp["min"] = float(round(grouped_df.groupby(keys_2).sum()["Valores"].min(),4)),
    df_tmp["total"] = float(round(grouped_df.groupby(keys_2).sum()["Valores"].sum(),4)),
    df_tmp["amplitude"] = df_tmp["max"] - df_tmp["min"]
    df_tmp["std"] = float(round(grouped_df.groupby(keys_2).sum()["Valores"].std(),4))
    # break
    data_feat = pd.concat([data_feat, df_tmp])



In [ ]:
values = [
    "count",
    "last",
    "mean",
    "max",
    "min",
    "total",
    "amplitude",
    "std"
]

data_feat_pivot = pd.pivot_table(data_feat, values=values, columns="grupo", index="cliente")
data_feat_pivot.columns = ['_'.join(col).strip() for col in data_feat_pivot.columns]
data_feat_pivot = data_feat_pivot.reset_index(drop=False)

In [ ]:
data_feat_pivot.columns

In [ ]:
# px.scatter(data_feat_pivot, x="total_Consumo", y="total_Injetada", hover_data="cliente")
sns.scatterplot(data_feat_pivot, x="total_Consumo", y="total_Injetada", palette="Blues")
plt.show()

### Get outliers

In [ ]:
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import Normalizer, MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler

In [ ]:
interest_columns = ["total_Consumo", "total_Injetada"]#, "total_TUSD"]
data_interest = data_feat_pivot[interest_columns]

In [ ]:

col_count = len(interest_columns)
fig, ax = plt.subplots(col_count, 1, figsize=(col_count*5, 6))
for i, col in enumerate(interest_columns):

    sns.histplot(data_interest, x=col, ax=ax[i])
plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
col_count = len(interest_columns)
fig, ax = plt.subplots(1, col_count, figsize=(12, 5))
for i, col in enumerate(interest_columns):
    sns.boxplot(data_interest, y=col, ax=ax[i])
plt.subplots_adjust(hspace=0.4)
plt.show()



IQR

In [ ]:
q1 = data_interest.quantile(0.25)
q3 = data_interest.quantile(0.75)
iqr = q3 - q1

outlier_mask = ((data_interest < (q1 - 1.5*iqr)) | (data_interest > (q3+1.5*iqr))).any(axis=1)
data_feat_pivot["out_iqr"] = 0
data_feat_pivot.loc[outlier_mask, "out_iqr"] = -1

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
sns.scatterplot(data_feat_pivot, x="total_Consumo", y="total_Injetada", hue="out_iqr", palette="Set1", ax=ax[0])
sns.scatterplot(data_feat_pivot, x="total_Consumo", y="total_TUSD", hue="out_iqr", palette="Set1", ax=ax[1])
sns.scatterplot(data_feat_pivot, x="total_Injetada", y="total_TUSD", hue="out_iqr", palette="Set1", ax=ax[2])
plt.subplots_adjust(wspace=0.4)
plt.show()



In [ ]:
X_data = data_interest.fillna(0)
clusterizer = DBSCAN(eps=100000,min_samples=10)
clusters = clusterizer.fit_predict(X_data)
data_feat_pivot["out_dbscan"] = clusters
print(data_feat_pivot["out_dbscan"].value_counts())
sns.scatterplot(data_feat_pivot, x="total_Consumo", y="total_Injetada", hue="out_dbscan", palette="Set1")
plt.show()


# Clusterização

In [ ]:
data_feat_pivot

In [ ]:
X = data_feat_pivot.loc[
    # :,
    (data_feat_pivot[["out_iqr", "out_dbscan"]]!=-1).all(axis=1),
    (data_feat_pivot.columns.str.contains("Consumo")) | (data_feat_pivot.columns.str.contains("Injetada")) | (data_feat_pivot.columns.str.contains("_TUSD"))
    ].fillna(0)
X

In [ ]:
# normalizer = Normalizer()
# normalizer = MaxAbsScaler()
# normalizer = MinMaxScaler()
normalizer = StandardScaler()
# normalizer = RobustScaler()
X_normalized = normalizer.fit_transform(X)

inertias = []
k_range = list(range(2, 11)) 
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_normalized)
    inertias.append(kmeans.inertia_)

plt.plot(k_range, inertias)
plt.ylabel("Inercias")
plt.xlabel("Numero Clusters")
plt.show()

In [ ]:
reduction = PCA(n_components=2, random_state=42)
X_reduced = reduction.fit_transform(X_normalized)
print(f"Explained Variance Ratio: {reduction.explained_variance_ratio_}")

k = 6 # based on elbon plot
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(X_normalized)
X["cluster"] = labels

df = pd.DataFrame(X_reduced)
df["cluster"] = labels
print(f"Clusters count")
print(df["cluster"].value_counts())
# Plot reduction
fig1, ax1 = plt.subplots(figsize=(10, 6))
sns.scatterplot(df, x=1, y=0, hue="cluster", palette="Set1", ax=ax1)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# plot original data
fig2, ax2 = plt.subplots(figsize=(10, 6))
sns.scatterplot(X, x="total_Consumo", y="total_Injetada", hue="cluster", palette="Set1", ax=ax2)
# plt.yscale("log")
# plt.xscale("log")
plt.show()

In [ ]:
data_feat_pivot = pd.merge(data_feat_pivot, X[["cluster"]], how="left", left_index=True, right_index=True)
data_feat_pivot["cluster"] = data_feat_pivot["cluster"].fillna(-1)
data_feat_pivot

In [ ]:
sns.boxplot(data_feat_pivot, x="cluster", y="mean_TUSD")

In [ ]:
sns.boxplot(X,x="cluster", y="total_Injetada")

In [ ]:
sns.boxplot(X,x="cluster", y="total_Consumo")

In [ ]:
X.loc[X["cluster"]==0]

In [ ]:
# normalizer = Normalizer()
# normalizer = MaxAbsScaler()
# normalizer = MinMaxScaler()
normalizer = StandardScaler()
# normalizer = RobustScaler()
X_normalized = normalizer.fit_transform(X)

reduction = PCA(n_components=2, random_state=42)
X_reduced = reduction.fit_transform(X_normalized)
print(f"Explained Variance Ratio: {reduction.explained_variance_ratio_}")

clusterizer = DBSCAN(eps=2)
labels = clusterizer.fit_predict(X_normalized)
X["cluster"] = labels

df = pd.DataFrame(X_reduced)
df["cluster"] = labels
print(f"Clusters count")
print(df["cluster"].value_counts())
# Plot reduction
fig1, ax1 = plt.subplots(figsize=(10, 6))
sns.scatterplot(df, x=1, y=0, hue="cluster", palette="Set1", ax=ax1)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# plot original data
fig2, ax2 = plt.subplots(figsize=(10, 6))
sns.scatterplot(X, x="total_Consumo", y="total_Injetada", hue="cluster", palette="Set1", ax=ax2)
# plt.yscale("log")
# plt.xscale("log")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Finding optimal number of clusters using the Elbow Method
inertias = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(range(1, 11), inertias, 'o-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_scaled = scaler.fit_transform(X)

# # Initialize and fit K-means
# kmeans = KMeans(n_clusters=3, random_state=0)
# clusters = kmeans.fit_predict(X)

# Fit Agglomerative Clustering
# agg_clustering = AgglomerativeClustering(n_clusters=2)
# agg_clusters = agg_clustering.fit_predict(X)

dbscan = DBSCAN(eps=2, min_samples=5)
db_clusters = dbscan.fit_predict(X_scaled)

x_cluster = X.copy()
x_cluster["cluster"] = db_clusters
var_list = X.columns


In [ ]:
x_cluster["cluster"].value_counts()

In [ ]:

for cltr in x_cluster["cluster"].unique():
    df_temp = x_cluster.loc[x_cluster["cluster"]==cltr]
    plt.figure(figsize=(10,4))
    sns.boxenplot(df_temp)
    plt.show()


In [ ]:

for clusters in clusters
sns.boxplot


In [ ]:
fig = px.histogram(data, x="Valores", color="Item Faturamento BW")
fig.show()

In [ ]:
for item in data["Item Faturamento BW"].unique():
    data_filtered = data.loc[data["Item Faturamento BW"] == item]
    plt.figure(figsize=(10,3))
    ax = sns.histplot(data_filtered, x="Valores")
    ax.set_title(item)
    plt.tight_layout()
    plt.show()

    

In [ ]:
for item in data["Item Faturamento BW"].unique():
    data_filtered = data.loc[data["Item Faturamento BW"] == item]

    # Create subplots (2 rows, 1 column)
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 6), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

    # Boxplot (First subplot)
    sns.boxplot(data=data_filtered, x="Meses", y="Valores", ax=axes[0])
    axes[0].set_title(f"{item}")

    # Remove X-axis markers from the boxplot
    axes[0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

    # Barplot (Second subplot) - Count occurrences of each month
    month_counts = data_filtered["Meses"].value_counts().reset_index()
    month_counts.columns = ["Meses", "Count"]
    barplot = sns.barplot(data=month_counts, x="Meses", y="Count", ax=axes[1])

    # Remove spines (contours)
    for spine in ["top", "right", "left", "bottom"]:
        axes[0].spines[spine].set_visible(False)
        axes[1].spines[spine].set_visible(False)

    # Add value labels to bars
    for p in barplot.patches:
        axes[1].annotate(f'{p.get_height()}', 
                         (p.get_x() + p.get_width() / 2., p.get_height()), 
                         ha='center', va='bottom', fontsize=10,)

    # axes[1].set_title(f"Count of Occurrences for {item}")

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
data

In [ ]:
item_occ = data[["Item Faturamento BW", "Meses", "Valores"]].groupby(["Item Faturamento BW", "Meses"]).count()
item_occ = item_occ.reset_index()
item_occ["Meses"] = item_occ["Meses"].astype(str)
item_occ


In [ ]:
px.line(item_occ,x="Meses", y="Valores", color="Item Faturamento BW")

In [ ]:
data

In [ ]:
data

In [ ]:
for (a, b,c,d), df  in data.groupby(["Empresa", "Classe de cálculo", "SubGrupo de Tensão", "Município"]):
    print(a)
    print(b)
    print(c)
    print(d)
    
    break


In [ ]:
df